In [ ]:
from pydantic import BaseModel, Field
from typing import Dict, List, Optional

class AnalysisSection(BaseModel):
    """Model for each analysis section"""
    key_points: List[str]
    summary: str

class AnalysisOutput(BaseModel):
    """Model for structured analysis output"""
    Revenue: Optional[AnalysisSection] = None
    Margins_and_Cashflow: Optional[AnalysisSection] = None
    Guidance: Optional[AnalysisSection] = None
    Capital_Allocation: Optional[AnalysisSection] = None
    MA: Optional[AnalysisSection] = None
    Inventory_and_Pricing_Strategy: Optional[AnalysisSection] = None
    Macro_Environment: Optional[AnalysisSection] = None
    Products_and_RD: Optional[AnalysisSection] = None
    confidence: float = Field(..., ge=0, le=1)

    class Config:
        allow_population_by_field_name = True

async def execute_prompts(self, prompts: List[Dict]) -> List[Dict]:
    """Execute prompts and return structured analysis"""
    responses = []
    
    for prompt in prompts:
        try:
            # Format the prompt to request structured output
            formatted_prompt = f"""
            Analyze the following aspects and provide a structured response with key points and summary.
            Each section should have:
            - A list of key points (up to 7 bullet points)
            - A brief summary
            
            {prompt["prompt"]}
            """
            
            response = JsonFormer(
                schema=AnalysisOutput,
                llm=self.llm
            ).invoke(formatted_prompt)
            
            # Convert pydantic model to dict and add metadata
            analysis_dict = response.dict()
            responses.append({
                "result": analysis_dict,
                "type": prompt["type"],
                "heading": prompt.get("heading", ""),
                "confidence": response.confidence,
                "bucket": prompt.get("bucket", 0)
            })
            
        except Exception as e:
            print(f"Error executing prompt: {e}")
            
    return responses

def _format_bucket_prompt(self, bucket: List[tuple], ect: str, html_prompt: str) -> str:
    """Format prompt for structured output"""
    metric_count = len(bucket)
    prompt = [
        f"Answer the following {metric_count} questions using content from the Earnings call text.",
        "For each section provide:",
        "- Key points (up to 7 bullet points)",
        "- A brief summary\n"
    ]
    
    for i, (heading, metrics) in enumerate(bucket, 1):
        metrics_str = ", ".join(metrics)
        prompt.append(
            f"{i}. Analyze the {metrics_str} of the company under the heading: {heading}"
        )
    
    prompt.append(f"\nEarnings call text:\n{ect}")
    
    return "\n".join(prompt)

In [ ]:
from pydantic import BaseModel, Field
from typing import Dict, List, Optional
from datetime import datetime

class ECTAnalyzer:
    # ... other methods remain the same ...

    async def get_ect_summary(self, company_id: str) -> List[Dict]:
        """
        Get ECT summary for a company.
        Returns a list containing the analysis or error message.
        """
        # Get documents
        documents_metadata = await self.get_earnings_by_company_id(company_id)
        if not documents_metadata:
            return [{"error": "No earnings document related to query is found"}]
            
        # Sort by date to get latest document
        documents_metadata.sort(
            key=lambda x: datetime.strptime(x.document_date_s, "%Y-%m-%d"),
            reverse=True
        )
        latest_doc = documents_metadata[0]
        
        # Get ECT content
        content = await self.get_earnings_content(latest_doc)
        if not content:
            return [{"error": "Could not fetch earnings content"}]
            
        # Generate and execute analysis prompts
        prompts = self.generate_ect_prompts(
            ect=content,
            id_bb_company=company_id,
            html_prompt=""  # Can be parameterized if needed
        )
        
        # Execute prompts and get structured analysis
        responses = await self.execute_prompts(prompts)
        
        # Process responses into final format
        analysis_sections = {}
        for response in responses:
            if response["confidence"] > 0.7:  # Confidence threshold
                # Each response now contains structured analysis sections
                analysis_dict = response["result"]
                analysis_sections.update(analysis_dict)
                
        # Return final structured response
        return [{
            "metadata": {
                "company": latest_doc.company_s,
                "period": f"Q{latest_doc.quarter_s} {latest_doc.year_s}",
                "date": latest_doc.document_date_s,
                "ticker": latest_doc.tickers_s[0] if latest_doc.tickers_s else None
            },
            "analysis": analysis_sections
        }]

# Usage example:
async def main():
    analyzer = ECTAnalyzer(gssso_token=gs_auth.get_gssso())
    result = await analyzer.get_ect_summary("107357")  # Example for Apple
    print(result)